## Primera implementación del método de involuciones 

In [1]:
using PyCall
using LaTeXStrings
@pyimport numpy as np
using SymPy
using Roots
using Plots

┌ Warning: `@pyimport foo` is deprecated in favor of `foo = pyimport("foo")`.
│   caller = _pywrap_pyimport(::PyObject) at PyCall.jl:410
└ @ PyCall /home/eve/.julia/packages/PyCall/BcTLp/src/PyCall.jl:410
┌ Info: Precompiling SymPy [24249f21-da20-56a4-8eb1-6a02cf4ae2e6]
└ @ Base loading.jl:1260


Definimos una función para componer n veces (tendremos 3 opciones)

In [2]:
composition(f, n) = ∘(ntuple(_ -> f, n)...) # donde f es una funcion de una variable y composition regresa una funcion

composition (generic function with 1 method)

In [3]:
compose(f, n) = reduce(∘, ntuple(_ -> f, n)) # donde f es una funcion de 1 variable

compose (generic function with 1 method)

In [4]:
function composeNT(f, n)
    function (x)  # <- Esto indica que es una funcion anonima
        val = f(x)
        for _ in 2:n
            val = f(val)
        end
        return val
    end
end


composeNT (generic function with 1 method)

In [5]:
function MapeoX(x,y,a,b)
    vx = x+a*(1-(y-b*sin(2*pi*x))^2)
    return vx
end

function MapeoY(x,y,a,b)
    vy = y-b*sin(2*pi*x)
    return vy
end

MapeoY (generic function with 1 method)

In [6]:
MapeoXA(y) = MapeoX(0.0,y,0.1,0.1)
MapeoYA(y) = MapeoY(0.0,y,0.1,0.1)

MapeoYA (generic function with 1 method)

In [7]:
Mapeo2XA(y) = MapeoXA(MapeoXA(y))
Mapeo2YA(y) = MapeoYA(MapeoYA(y))

Mapeo2YA (generic function with 1 method)

In [8]:
Mapeo3XA(y) = MapeoXA(Mapeo2XA(y))
Mapeo3YA(y) = MapeoYA(Mapeo2YA(y))

Mapeo3YA (generic function with 1 method)

In [20]:
Mapeo3XA

Mapeo3XA (generic function with 1 method)

In [17]:
#using NLsolve

function f!(F, x)
    F[1] = (x[1]+3)*(x[2]^3-7)+18
    F[2] = sin(x[2]*exp(x[1])-1)
end

┌ Info: Precompiling NLsolve [2774e3e8-f4cf-5e23-947b-6d7e65073b56]
└ @ Base loading.jl:1260


f! (generic function with 1 method)

In [18]:
function j!(J, x)
    J[1, 1] = x[2]^3-7
    J[1, 2] = 3*x[2]^2*(x[1]+3)
    u = exp(x[1])*cos(x[2]*exp(x[1])-1)
    J[2, 1] = x[2]*u
    J[2, 2] = u
end

nlsolve(f!, j!, [ 0.1; 1.2])

Results of Nonlinear Solver Algorithm
 * Algorithm: Trust-region with dogleg and autoscaling
 * Starting Point: [0.1, 1.2]
 * Zero: [-3.7818049096324184e-16, 1.0000000000000002]
 * Inf-norm of residuals: 0.000000
 * Iterations: 4
 * Convergence: true
   * |x - x'| < 0.0e+00: false
   * |f(x)| < 1.0e-08: true
 * Function Calls (f): 5
 * Jacobian Calls (df/dx): 5

In [12]:
function f!(vec1,vec2)
    x = vec1[1]
    y = vec1[2]
    vec2[1] = Mapeo3XA(y)
    vec2[2] = Mapeo3YA(y)
end

f! (generic function with 1 method)

In [13]:
cerosA = nsolve(f!,[0.3;0.3])

MethodError: MethodError: no method matching nsolve(::typeof(f!), ::Array{Float64,1})
Closest candidates are:
  nsolve(!Matched::SymPy.SymbolicObject, ::Any...; kwargs...) at /home/eve/.julia/packages/SymPy/fI5pg/src/importexport.jl:120

In [ ]:
function periodicoSimY(fx, fy, intervalo)
    cerosy1 = find_zeros(fx,intervalo[1],intervalo[2])
    cerosy2 = find_zeros(fy,intervalo[1],intervalo[2])
    return [cerosy1,cerosy2]
end

In [ ]:
cerosY = periodicoSimY(Mapeo2XA, Mapeo2YA, [-4.0,4.0])

In [ ]:
function simetria1(y,a)
    x = mod((a/2)*(1-y^2),1)
    return x
end

In [ ]:
simetria1A(y) = simetria1(y,0.1)

In [ ]:
function periodicoSimX(fx,fy,simetria,cerosy1,cerosy2)
    cerosx1 = [simetria(cerosy1[i]) for i in 1:length(cerosy1)]
    cerosx2 = [simetria(cerosy2[i]) for i in 1:length(cerosy2)]
    return [cerosx1, cerosx2]
end

In [ ]:
periodicoSimX(Mapeo2XA,Mapeo2YA,simetria1A,cerosY[1],cerosY[2])

In [ ]:
mapeo2X(x)=composition(MapeoXA,20)(x) #esto esta mal debe ser funcion de y 
mapeo2Y(y)=composition(MapeoYA,20)(y)

In [ ]:
cerosY = periodicoSimY(mapeo2X,mapeo2Y,[-18.0,18.0])

In [ ]:
periodicoSimX(mapeo2X,mapeo2Y,simetria1A,cerosY[1],cerosY[2])

* Como en general vamos a dar un mapeo y queremos que se sustituyan las simetrias y se vayan encontrando los puntos periodicos queremos juntar lo anterior 

Poe ejemplo imaginemos que tenemos el mapeo siguiente

In [ ]:
function MapeoX(x,y,a,b)
    vx = x+a*(1-(y-b*sin(2*pi*x))^2)
    return vx
end

function MapeoY(x,y,a,b)
    vy = y-b*sin(2*pi*x)
    return vy
end

* Despues nos dan los parametros y la simetria 

a=0.1
b=0.1

sim1 = funcion de y


* Necesitamos hacer un nuevo mapeo interno con esas caracteristicas


In [ ]:
function substitutionMap(fx,fy,a,b,sim)
    MapeoXA(y) = MapeoX(sim(y),y,a,b)
    MapeoYA(y) = MapeoY(sim(y),y,a,b)
    return [MapeoXA,MapeoYA]
end

In [ ]:
simetria1(y)= 0.0
simetria2(y)=(1-y^2)/2

In [ ]:
Maps1 = substitutionMap(MapeoX,MapeoY,0.1,0.1,simetria1)

In [ ]:
Maps1[2]

* Estas funciones son las que iteramos despues 


In [ ]:
mapeo2X(y)=composition(Maps1[1],20)(y)
mapeo2Y(y)=composition(Maps1[2],20)(y)

In [ ]:
cerosY = periodicoSimY(mapeo2X,mapeo2Y,[-18.0,18.0])

In [ ]:
periodicoSimX(mapeo2X,mapeo2Y,simetria1,cerosY[1],cerosY[2])

* Que pasa si la condicion de simetria es nua funcion?

* Veamos si se puede sustituir

In [ ]:
Maps2 = substitutionMap(MapeoX,MapeoY,0.1,0.1,simetria2)

In [ ]:
mapeo3X(y)=composition(Maps2[1],20)(y)
mapeo3Y(y)=composition(Maps2[2],20)(y)

In [ ]:
cerosY2 = periodicoSimY(mapeo3X,mapeo3Y,[-1.0,1.0])

In [ ]:
periodicoSimX(mapeo3X,mapeo3Y,simetria2,cerosY2[1],cerosY2[2])

* Ahora juntamos todo en una funcion o dos

In [ ]:
function PeriodicosS(fx,fy,simetria,a,b,n,intervalo)
    Maps = substitutionMap(fx,fy,a,b,simetria)
    mapeoX(y)=composition(Maps[1],n)(y)
    mapeoY(y)=composition(Maps[2],n)(y)
    cerosY = periodicoSimY(mapeoX,mapeoY,[intervalo[1],intervalo[2]])
    cerosX = periodicoSimX(mapeoX,mapeoY,simetria,cerosY[1],cerosY[2])
    return [cerosX,cerosY]
end

In [ ]:
PeriodicosS(MapeoX,MapeoY,simetria2,0.1,0.1,20,[-1,1])

In [ ]:
PeriodicosS(MapeoX,MapeoY,simetria1,0.1,0.1,20,[-19,19])

In [ ]:
push!(LOAD_PATH,"/home/eve/Documents/Matematicas/Proyectos/Implementacion")

In [ ]:
using PeriodicosSymmetries

In [ ]:
PeriodicosS(MapeoX,MapeoY,simetria1,0.1,0.1,20,[-19,19])

In [ ]:
?PeriodicosS

In [ ]:
function MapeoX(x,y,a,b)
    vx = x+a*(1-(y-b*sin(2*pi*x))^2)
    return vx
end

function MapeoY(x,y,a,b)
    vy = y-b*sin(2*pi*x)
    return vy
end

In [ ]:
simetria1(y)= 0.0
simetria2(y)=(1-y^2)/2